In [2]:
import torchtext

## 读入数据

In [3]:
path='./data/src-train.txt'

In [4]:
lists=open(path,'r',encoding='utf-8').readlines()

In [5]:
lists[:3]

['It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .\n',
 'Federal Master Trainer and Senior Instructor of the Italian Federation of Aerobic Fitness , Group Fitness , Postural Gym , Stretching and Pilates; from 2004 , he has been collaborating with Antiche Terme as personal Trainer and Instructor of Stretching , Pilates and Postural Gym .\n',
 '&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .\n']

## Fields 对象

In [6]:
from  torchtext.data import Field

In [7]:
tok=lambda x: x.split(' ')

In [8]:
TEXT=Field(tokenize=tok,eos_token='<eos>',init_token='<sos>',include_lengths=True)

In [10]:
#可以直接使用preprocess 方法处理一个字符串
s=TEXT.preprocess(lists[0])
s

['It',
 'is',
 'not',
 'acceptable',
 'that',
 ',',
 'with',
 'the',
 'help',
 'of',
 'the',
 'national',
 'bureaucracies',
 ',',
 'Parliament',
 '&apos;s',
 'legislative',
 'prerogative',
 'should',
 'be',
 'made',
 'null',
 'and',
 'void',
 'by',
 'means',
 'of',
 'implementing',
 'provisions',
 'whose',
 'content',
 ',',
 'purpose',
 'and',
 'extent',
 'are',
 'not',
 'laid',
 'down',
 'in',
 'advance',
 '.']

## 生成Examples 样本
### dataset 传入的参数有两个dataset(examples,filds)

In [11]:
examples=[]

In [12]:
for  x in lists:
    e=torchtext.data.Example()
    setattr(e,'text',TEXT.preprocess(x))
    examples.append(e)

In [15]:
examples[0]

## 给filds建立字典

In [16]:
from collections import Counter

In [27]:
TEXT.__dict__.keys()

dict_keys(['sequential', 'use_vocab', 'init_token', 'eos_token', 'unk_token', 'fix_length', 'tensor_type', 'preprocessing', 'postprocessing', 'lower', 'tokenize', 'include_lengths', 'batch_first', 'pad_token', 'pad_first', 'vocab'])

In [17]:
c=Counter()
for e in examples:
    c.update(e.text)

In [50]:
specials=[TEXT.unk_token,TEXT.pad_token,TEXT.init_token,TEXT.eos_token]   # 请把unk_token 放第一位，因为源码会直接把unk词映射到0
specials

['<unk>', '<pad>', '<sos>', '<eos>']

In [51]:
TEXT.vocab=TEXT.vocab_cls(c,max_size=5000,specials=specials)

In [52]:
len(TEXT.vocab)

5004

In [53]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             'the': 4,
             ',': 5,
             '.': 6,
             'of': 7,
             'and': 8,
             'to': 9,
             'in': 10,
             'a': 11,
             'is': 12,
             'that': 13,
             'for': 14,
             'on': 15,
             'The': 16,
             'with': 17,
             'be': 18,
             'are': 19,
             'I': 20,
             'this': 21,
             'as': 22,
             'we': 23,
             'have': 24,
             'it': 25,
             'not': 26,
             'you': 27,
             'by': 28,
             'will': 29,
             'from': 30,
             'which': 31,
             'has': 32,
             'at': 33,
             'an': 34,
             'or': 35,
             '-': 36,
             'can': 37,
             'our': 38,
             '"': 39,


## 建立Dataset

In [54]:
dataset=torchtext.data.Dataset(examples,{'text':TEXT})

In [55]:
dataset.__dict__.keys()

dict_keys(['examples', 'fields'])

In [56]:
len(dataset)

10000

## 迭代

In [57]:
ite=torchtext.data.BucketIterator(dataset,batch_size=32,sort_key=lambda x:len(x.text),sort_within_batch=True,device=-1)

In [64]:
Iter=iter(ite)

In [65]:
next(Iter)

In [66]:
batch=next(Iter)

In [67]:
batch.__dict__.keys()

dict_keys(['batch_size', 'dataset', 'train', 'text'])

In [68]:
batch.text[0]

Variable containing:
    2     2     2  ...      2     2     2
    0   659   752  ...     47   320  3066
   18     7     5  ...   1740    12     4
       ...          ⋱          ...       
 2439  2762  2940  ...     73   741  2279
    6     6     6  ...      6     6     6
    3     3     3  ...      3     3     3
[torch.LongTensor of size 36x32]